# Import packages 

In [7]:
import cv2
import os
from keras.preprocessing import image
import matplotlib.pyplot as plt
import random
import numpy as np

In [6]:
!pip install keras

# Data Preparation

In [8]:
categories=["with mask","without mask"]

In [10]:
data=[]
for category in categories:
  path=os.path.join('C:\\Users\HP\Downloads\FinalDet\dataset',category)
  label=categories.index(category)
  for file in os.listdir(path):
    img_path=os.path.join(path,file)
    # print(img_path)
    img=cv2.imread(img_path)
    img=cv2.resize(img,(224,224))
    # print(img.shape)
    data.append([img,label])

In [11]:
len(data)

1376

In [12]:
random.shuffle(data)

In [13]:
X=[]
Y=[]
for features,label in data:
  X.append(features)
  Y.append(label)

In [15]:
X=np.array(X)
Y=np.array(Y)
print(X.shape)
print(Y.shape)

(1376, 224, 224, 3)
(1376,)


# Data Preprocessing 

In [16]:
X=X/255

In [17]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.2)

# Model Customization

In [18]:
from keras.applications.vgg16 import VGG16

In [19]:
vgg=VGG16()

553467904/553467096 [==============================] - 599s 1us/step


In [20]:
vgg.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [21]:
from keras import Sequential
model=Sequential()

In [22]:
for layer in vgg.layers[:-1]:
  model.add(layer)

In [23]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)       2

In [24]:
for layer in model.layers:
  layer.trainable=False

In [25]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)       2

In [26]:
from keras.layers import Dense
model.add(Dense(1,activation='sigmoid'))

In [27]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)       2

# Training The Model

In [28]:
model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])
model.fit(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Epoch 1/5
35/35 [==============================] - 589s 16s/step - loss: 0.6881 - accuracy: 0.5885 - val_loss: 0.3981 - val_accuracy: 0.8841
Epoch 2/5
35/35 [==============================] - 316s 9s/step - loss: 0.3733 - accuracy: 0.9084 - val_loss: 0.2869 - val_accuracy: 0.9275
Epoch 3/5
35/35 [==============================] - 311s 9s/step - loss: 0.2590 - accuracy: 0.9409 - val_loss: 0.2290 - val_accuracy: 0.9239
Epoch 4/5
35/35 [==============================] - 310s 9s/step - loss: 0.2274 - accuracy: 0.9420 - val_loss: 0.1901 - val_accuracy: 0.9529
Epoch 5/5
35/35 [==============================] - 314s 9s/step - loss: 0.1919 - accuracy: 0.9404 - val_loss: 0.1800 - val_accuracy: 0.9493


# Model Evaluation

In [30]:
def detect_face_mask(img):
    y_pred=model.predict_classes(img.reshape(1,224,224,3))
    return y_pred[0][0]

In [48]:
def draw_lebel(img,text,pos,bg_color):
    text_size=cv2.getTextSize(text,cv2.FONT_HERSHEY_SIMPLEX,1,cv2.FILLED)
    end_x=pos[0]+text_size[0][0]+2
    end_y=pos[1]+text_size[0][1]-2
    cv2.rectangle(img,pos,(end_x,end_y),bg_color,cv2.FILLED)
    cv2.putText(img,text,pos,cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),1,cv2.LINE_AA)

In [40]:
# haar=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')


In [41]:
# def detect_face(img):
#     coods=haar.detectMultiScale(img)
#     return coods

# Real time Model evaluation

In [50]:
cap=cv2.VideoCapture(0)
while True:
    ret,frame=cap.read()
    img=cv2.resize(frame,(224,224))
    y_pred=detect_face_mask(img)
#     coods=detect_face(cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY))
#     for x,y,w,h in coods:
#         cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),3)
    if y_pred==0:
        draw_lebel(frame,"Mask",(100,100),(0,255,0))
    else:
        draw_lebel(frame,"No Mask",(100,100),(0,0,255))
    cv2.imshow("window",frame)
    if cv2.waitKey(1) & 0xFF==ord('q'):
        break
cv2.destroyAllWindows()